In [21]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(".."))

from sklearn.model_selection import train_test_split
from src.data.data_generator import generate_classpass_data
from src.models.booking_prediction import BookingLikelihoodModel
from src.models.churn_model import ChurnModel
from src.pricing.dynamic_pricing import DynamicPricingEngine
from src.preprocessing import preprocess_features
from train_model import train_booking_model, train_churn_model
from utils import save_model, load_model

# load data

df = generate_classpass_data(50000)
df = pd.read_csv("../data/classpass_data.csv")

In [24]:
# preprocess features

X, y = preprocess_features(df, target = "booked")

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [ ]:
# train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# train bookinglikelihood model

booking_model = train_booking_model(X_train, y_train)

save_model(booking_model, "../models/booking_model.pkl")

In [ ]:
# evaluate model

pred_probs = booking_model.predict_proba(X_test)[:,1]

print("Booking AUC:", roc_auc_score(y_test, pred_probs))

In [ ]:
# train churn model

X_churn, y_churn = preprocess_features(df, target = "churned")

X_train_c, X_test_c, y_train_c, y_test_c, = train_test_split(X_churn, y_churn,test_size = 0.2, random_state = 42)

churn_model = train_churn_model(X_train_c, y_train_c)

save_model(churn_model, "../models/churn_model.pkl")

pred_churn_probs = churn_model.predict_proba(X_test_c)[:,1]

print("Churn AUC:", roc_auc_score(y_test_c, pred_churn_probs))